In [1]:
import functools
import enum
import os

from BH.data_loader import *
from BH.generate_data import *
from training_info import *
# from Model_e import Model_e,Direction,Reduction
from Train import train,print_accuracies
from torch_geometric.loader import DataLoader


os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device ="cuda:0"
use_pretrained_weights = True  #@param{type:"boolean"}
hold_graphs_in_memory = False  #@param{type:"boolean"}

gb = 1024**3
total_memory = psutil.virtual_memory().total / gb
if total_memory < 20 and hold_graphs_in_memory:
    raise RuntimeError(f"It is unlikely your machine (with {total_memory}Gb) will have enough memory to complete the colab's execution!")

print("Loading input data...")
full_dataset, train_dataset, test_dataset = load_input_data(DIR_PATH)

Loading input data...
Generating data from the directory /Data/Ptab/n=7_2row


In [2]:
from torch_geometric.data import Data
from torch.utils.data import Dataset
from torch_geometric.loader import DataLoader
import torch
class CustomDataset(Dataset):
    def __init__(self, input_data):
        self.features = input_data.features
        self.labels = input_data.labels
        self.rows = input_data.rows
        self.cols = input_data.columns
        self.edge_types = input_data.edge_types

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        edge_index = torch.tensor([self.rows[idx], self.cols[idx]], dtype=torch.long)
        return Data(x=torch.from_numpy(self.features[idx]).float(), edge_index=edge_index, 
             edge_types = torch.tensor(self.edge_types[idx][:, np.newaxis], dtype=torch.float),
             y=torch.from_numpy(np.array(self.labels[idx])))

In [3]:
node_dim=64
edge_dim=8
graph_deg=7
batch_size=32

test_dataset = CustomDataset(test_dataset)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

train_dataset = CustomDataset(train_dataset)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [4]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATv2Conv,GCNConv


class GCN_single(torch.nn.Module):
    def __init__(self,num_edge_types,depth):
        super().__init__()
        self.num_edge_types= num_edge_types
        self.depth=depth
        self.node_linear = torch.nn.Linear(1,node_dim)
        self.edge_linear = torch.nn.Linear(1,edge_dim)
        self.conv1 = GCNConv(node_dim, node_dim)
        self.conv2 = GCNConv(node_dim, node_dim)
        self.conv3 = GCNConv(node_dim, node_dim)
        self.conv4 = GCNConv(node_dim, node_dim)
        
        self.out1 = torch.nn.Linear(node_dim,node_dim)
        self.out2 = torch.nn.Linear(node_dim,1)
        self.initialize_parameters()

    def initialize_parameters(self):
        for module in self.modules():
            if isinstance(module, torch.nn.Linear):
                torch.nn.init.xavier_uniform_(module.weight)
                if module.bias is not None:
                    torch.nn.init.zeros_(module.bias)
        

    def forward(self, data):
        x, edge_index, edge_types = data.x, data.edge_index, data.edge_types
        x = self.node_linear(x)        
        for i in range(self.depth):
            mask1 = (edge_types == 1)
            edge_index1 = edge_index[:, mask1.squeeze()]
            x1 = self.conv1(x, edge_index1)
            x1 = F.relu(x1)
            
            mask2 = (edge_types == 2)
            edge_index2 = edge_index[:, mask2.squeeze()]
            x2 = self.conv2(x, edge_index2)
            x2 = F.relu(x2)
            
            mask3 = (edge_types == 3)
            edge_index3 = edge_index[:, mask3.squeeze()]
            x3 = self.conv3(x, edge_index3)
            x3 = F.relu(x3)
            
            mask4 = (edge_types == 4)
            edge_index4 = edge_index[:, mask4.squeeze()]
            x4 = self.conv4(x, edge_index4)
            x4 = F.relu(x4)
            
            x = x1+x2+x3+x4
            x = F.relu(x)
        xx = torch.reshape(x,(-1,graph_deg,node_dim))
        xxx,_ = torch.max(xx,dim=1)
        xxx = self.out1(xxx)
        xxx = self.out2(xxx)
        
        return xxx
#         return F.log_softmax(xxx, dim=1)
    
class GCN_multi(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.num_edge_types=4
        self.depth =5
        self.GCN_single = GCN_single(self.num_edge_types, self.depth)        

    def forward(self, data,T=1):
        batch = data.batch
        batch = batch[::5]
        x = self.GCN_single(data)
        x=torch.sigmoid(x/T)
        unique_batches = torch.unique(batch)
        
#         sums_tensor = torch.zeros(len(unique_batches), requires_grad=True)

#         # Loop over unique batches and add elements of 'x' within each batch directly to sums_tensor
#         for i, ub in enumerate(unique_batches):
#             sums_tensor[i] = x[batch == ub].sum()

            
            # Loop over unique batches and sum elements of 'x' within each batch
        sum_list = [x[batch == ub].sum() for ub in unique_batches]

        # Stack list of sums to create a tensor
        sums_tensor = torch.stack(sum_list)
#         print(sums_tensor)
        
#         unique_batches = torch.unique(batch)
#         sums = []
#         for ub in unique_batches:
#             sums.append(x[batch == ub].sum())

#         # Convert list of sums to tensor
#         sums_tensor = torch.tensor(sums)
        
        return sums_tensor

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = pastGCN().to(device)
model = GCN_multi().to(device)
# data = batch.to(device)
# torch.nn.init.xavier_normal(model)
loss_function = torch.nn.CrossEntropyLoss()
loss_function = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)

In [6]:
test_dataset[2].y

/tmp/ipykernel_1409624/1906490525.py:17: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343970094/work/torch/csrc/utils/tensor_new.cpp:245.)
  edge_index = torch.tensor([self.rows[idx], self.cols[idx]], dtype=torch.long)


tensor(0)

In [7]:
test_dataset[2].edge_index

tensor([[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 4, 4, 4, 4, 5, 5,
         5, 5, 6, 6],
        [0, 1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 2, 5, 6, 2, 3, 2, 3, 4, 6, 3, 4,
         5, 6, 3, 6]])

In [15]:
max(full_dataset.labels)

48

In [16]:
for data in test_loader:
    break
print(data)

DataBatch(x=[532, 1], edge_index=[2, 2128], y=[32], edge_types=[2128, 1], batch=[532], ptr=[33])


In [17]:
data.batch[::5]

tensor([ 0,  0,  0,  0,  0,  0,  1,  2,  2,  3,  4,  4,  5,  5,  6,  6,  6,  6,
         6,  6,  7,  8,  8,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9, 10, 11,
        11, 12, 13, 13, 14, 14, 14, 14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15,
        15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 16, 16, 16,
        17, 18, 18, 19, 20, 21, 21, 22, 23, 23, 24, 25, 26, 26, 26, 27, 27, 27,
        27, 27, 27, 27, 28, 28, 28, 28, 28, 28, 28, 29, 29, 30, 31, 31, 31])

In [18]:
num_epochs=1000
for epoch in range(num_epochs):
    # Training phase
    model.train()
    for batch in train_loader:
        batch.to(device)
        optimizer.zero_grad()
        out = model(batch)
        
        batch.y = batch.y.float()
        loss = loss_function(out, batch.y)
        loss.backward()
        optimizer.step()
    print(loss)
    
    # Evaluation phase
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in train_loader:
            batch.to(device)
            outputs = model(batch)
#             _,predicted = torch.max(outputs.data, 1)
            predicted = outputs
            total += batch.y.size(0)
#             correct += (predicted == batch.y).sum().item()
            correct += ((predicted - batch.y)**2<0.1).sum().item()

    # Compute accuracy
    accuracy = correct / total

    print("Epoch [{}/{}], Accuracy: {:.2%}".format(epoch + 1, num_epochs, accuracy))

IndexError: The shape of the mask [93] at index 0 does not match the shape of the indexed tensor [66, 1] at index 0

In [11]:
loss

tensor(0.0006, grad_fn=<MseLossBackward0>)

In [12]:
out.shape

torch.Size([16])

In [13]:
test_dataset[0]

Data(x=[5, 1], edge_index=[2, 15], y=1, edge_types=[15, 1])

In [14]:
batch.x[0]

tensor([1.])

In [15]:
batch.y

tensor([1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1])

In [16]:
batch

DataBatch(x=[85, 1], edge_index=[2, 255], y=[16], edge_types=[255, 1], batch=[85], ptr=[17])

In [17]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for batch in train_loader:
        batch.to(device)
        outputs = model(batch)
        _, predicted = torch.max(outputs.data, 1)
        total += batch.y.size(0)
        correct += (predicted == batch.y).sum().item()
        
# Compute accuracy
accuracy = correct / total

print("Epoch [{}/{}], Accuracy: {:.2%}".format(epoch + 1, num_epochs, accuracy))

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [ ]:
from datetime import datetime
current_date = datetime.now().strftime('%Y%m%d')

# Define the path to save the model parameters
# You might want to modify this to a directory of your choice
path = f"./model_parameters_{current_date}.pth"

# Save the model parameters
torch.save(model.state_dict(), path)